<a href="https://colab.research.google.com/github/fedorov/idc-ghc-xfersyntax-testing/blob/main/IDC_J2K_debugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/ImagingDataCommons/IDC-ProjectManagement/issues/2044

In [ ]:

#@title Enter your Project ID
# initialize this variable with your Google Cloud Project ID!
my_ProjectID = "idc-sandbox-000" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = my_ProjectID

from google.colab import auth
auth.authenticate_user()

AuthorizationError: Error fetching credentials

In [4]:
!pip install --upgrade idc-index

  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached google_cloud_core-2.5.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_resumable_media-2.8.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached googleapis_common_protos-1.72.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached proto_plus-1.27.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached grpcio-1.76.0-cp313-cp313-macosx_11_0_universal2.whl.metadata (3.7 kB)
  Using cached grpcio_status-1.76.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached google_crc32c-1.8.0-cp313-cp313-macosx_12_0_arm64.whl.metadat

In [18]:
!pip install pydicom

  Using cached pydicom-3.0.1-py3-none-any.whl.metadata (9.4 kB)
Using cached pydicom-3.0.1-py3-none-any.whl (2.4 MB)


In [60]:
# DICOMweb endpoints

DICOMWEB_ENDPOINTS = { \
    "IDC Proxy": "https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb", 
    "Google Public Store v23": "https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v23/dicomWeb",
    "Google Public Store v22": "https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v22/dicomWeb",
    "IDC Internal Store": "https://healthcare.googleapis.com/v1/projects/canceridc-data/locations/us/datasets/idc/dicomStores/v23/dicomWeb"
}

In [15]:
from idc_index import IDCClient

client = IDCClient()

client.fetch_index("sm_instance_index")

2026-01-14 11:52:54,006 - Index sm_instance_index already installed, loading from /Users/af61/Library/Application Support/idc_index_data/23.2.8/sm_instance_index.parquet


In [ ]:
DOWNLOADED_INSTANCES_DIR = "./gcs_downloaded"

In [56]:
from pathlib import Path
import pydicom as pd

class TestSMSample:
    def __init__(self, label, study, series, instance, frame, downloadDir=DOWNLOADED_INSTANCES_DIR):
        self.label = label
        self.study = study
        self.series = series
        self.instance = instance
        self.frame = frame

        self.idc_client = IDCClient()
        self.idc_client.fetch_index("sm_instance_index")
        self.downloadDir = downloadDir
        self.instance_path = None

    def __getitem__(self, key):
        return getattr(self, key)
    
    def __str__(self):
        details = f"TestSMSample(study={self.study}, series={self.series}, instance={self.instance}, frame={self.frame})"
        xfer_syntax = self.get_local_xfer_syntax()
        details += f"\nTransfer Syntax:  {xfer_syntax} {str(xfer_syntax.name)}"        
        return details

    def download_instance(self, downloadDir=None):
        if downloadDir is None:
            downloadDir = self.downloadDir
        # create downloadDir if it does not exist
        directory_path = Path(downloadDir)
        directory_path.mkdir(parents=True, exist_ok=True)

        self.idc_client.download_dicom_instance(self.instance, downloadDir=downloadDir, dirTemplate="")

        # map SOPInstanceUID to crdc_instance_uid
        crdc_instance_uid = self.idc_client.sm_instance_index[
            self.idc_client.sm_instance_index['SOPInstanceUID'] == self.instance
        ]['crdc_instance_uuid']
        self.instance_path = Path(downloadDir) / f"{crdc_instance_uid.values[0]}.dcm"

    def get_local_xfer_syntax(self, downloadDir=None):
        if downloadDir is None:
            downloadDir = self.downloadDir
        if self.instance_path is None or not self.instance_path.exists():
            self.download_instance(downloadDir)
        local_file_path = self.instance_path
        dcm = pd.dcmread(local_file_path)
        return dcm.file_meta.TransferSyntaxUID


In [17]:
failing_slide.download_instance(DOWNLOADED_INSTANCES_DIR)

2026-01-14 11:53:06,210 - Disk size needed: 4.3 MB
2026-01-14 11:53:06,211 - Disk size available: 2.8 GB
2026-01-14 11:53:06,569 - Not using s5cmd sync as the destination folder is empty or sync or progress bar is not requested
2026-01-14 11:53:06,587 - Initial size of the directory: 0 bytes
2026-01-14 11:53:06,588 - Approximate size of the files that need to be downloaded: 4.3 MB
2026-01-14 11:53:08,268 - Successfully downloaded files to /Users/af61/github/idc-ghc-xfersyntax-testing/gcs_downloaded


In [57]:
test_samples = []

test_samples.append(TestSMSample("failing slide", "2.25.302737996345872783571112300080988167697",
                 "1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0",
                 "1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0",
                 "56"))

test_samples.append(TestSMSample("succeeding slide",
                    "2.25.205318147612807799490440393069389220550",
                    "1.3.6.1.4.1.5962.99.1.826406969.1146508888.1727403292729.4.0",
                    "1.3.6.1.4.1.5962.99.1.826406969.1146508888.1727403292729.38.0",
                    "64"))

2026-01-14 15:08:44,419 - Index sm_instance_index already installed, loading from /Users/af61/Library/Application Support/idc_index_data/23.2.8/sm_instance_index.parquet
2026-01-14 15:08:48,261 - Index sm_instance_index already installed, loading from /Users/af61/Library/Application Support/idc_index_data/23.2.8/sm_instance_index.parquet


2026-01-14 14:42:52,329 - Disk size needed: 4.3 MB
2026-01-14 14:42:52,330 - Disk size available: 2.57 GB
2026-01-14 14:42:53,028 - Not using s5cmd sync as the destination folder is empty or sync or progress bar is not requested
2026-01-14 14:42:53,071 - Initial size of the directory: 2.15 MB
2026-01-14 14:42:53,074 - Approximate size of the files that need to be downloaded: 4.3 MB
2026-01-14 14:42:54,610 - Successfully downloaded files to /Users/af61/github/idc-ghc-xfersyntax-testing/gcs_downloaded


TestSMSample(study=2.25.302737996345872783571112300080988167697, series=1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0, instance=1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0, frame=56)
Transfer Syntax:  1.2.840.10008.1.2.4.91 + (JPEG 2000 Image Compression)


In [ ]:
ACCEPT_HEADERS = {
  "slim": "-H 'Accept: multipart/related; type=\"image/jls\"; transfer-syntax=1.2.840.10008.1.2.4.80, multipart/related; type=\"image/jls\"; transfer-syntax=1.2.840.10008.1.2.4.81, multipart/related; type=\"image/jp2\"; transfer-syntax=1.2.840.10008.1.2.4.90, multipart/related; type=\"image/jp2\"; transfer-syntax=1.2.840.10008.1.2.4.91, multipart/related; type=\"image/jpx\"; transfer-syntax=1.2.840.10008.1.2.4.92, multipart/related; type=\"image/jpx\"; transfer-syntax=1.2.840.10008.1.2.4.93, multipart/related; type=\"application/octet-stream\"; transfer-syntax=*, */*;q=0.1'",
  ".91 only": "-H 'Accept: multipart/related; type=\"image/jp2\"; transfer-syntax=1.2.840.10008.1.2.4.91'"
}

def build_url(store, slide):
  return f"{store}/studies/{slide.study}/series/{slide.series}/instances/{slide.instance}/frames/{slide.frame}"

def build_curl_command(store, slide, accept_headers=""):
    url = build_url(store, slide)
    auth = "-H \"Authorization: Bearer $(gcloud auth print-access-token)\""
    command = f"curl -sS -X GET {auth} {accept_headers} {url}"
    return command

curl_command = build_curl_command(GOOGLE_PUBLIC_STORE_v22, test_samples[0], slim_accept_headers)
print("Curl command to download failing slide:")
print(curl_command) 

Curl command to download failing slide:
curl -sS -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" -H 'Accept: multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.80, multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.81, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.90, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.91, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.92, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.93, multipart/related; type="application/octet-stream"; transfer-syntax=*, */*;q=0.1' https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v22/dicomWeb/studies/2.25.302737996345872783571112300080988167697/series/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0/instances/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0/fra

In [85]:
import subprocess

for slide in test_samples:
    print("\n" + str(slide))

    print("\nCurl commands to download slide from various endpoints:")
    for endpoint in DICOMWEB_ENDPOINTS:
        print(f"\t{endpoint}:")
        store = DICOMWEB_ENDPOINTS[endpoint]
        curl_command = build_curl_command(store, slide, slim_accept_headers)
        print(curl_command)
        # run the command and print out result
        subprocess.run(curl_command, shell=True)




TestSMSample(study=2.25.302737996345872783571112300080988167697, series=1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0, instance=1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0, frame=56)
Transfer Syntax:  1.2.840.10008.1.2.4.91 JPEG 2000 Image Compression

Curl commands to download slide from various endpoints:
	IDC Proxy:
curl -sS -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" -H 'Accept: multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.80, multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.81, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.90, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.91, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.92, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.93, multipart/related; type="application/octet-stream"; transfer-syntax=*, */*;q=0.1' https://pr

In [ ]:
# claude.ai generated - need more work

import subprocess
import json
import re
from itertools import product
from datetime import datetime
import pandas as pd

# =============================================================================
# CONFIGURATION: Define all test parameters
# =============================================================================

# DICOMWeb Endpoints to test
DICOMWEB_ENDPOINTS = {
    "IDC_PROXY": "https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb",
    "GOOGLE_PUBLIC_v23": "https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v23/dicomWeb",
    "GOOGLE_PUBLIC_v22": "https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v22/dicomWeb",
}

# Accept Headers to test
ACCEPT_HEADERS = {
    "SLIM_FULL": 'multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.80, multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.81, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.90, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.91, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.92, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.93, multipart/related; type="application/octet-stream"; transfer-syntax=*, */*;q=0.1',
    "JPEG2000_ONLY": 'multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.90, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.91',
    "OCTET_STREAM": 'multipart/related; type="application/octet-stream"; transfer-syntax=*',
    "NO_HEADER": "",
}

# =============================================================================
# TEST EXECUTION FUNCTIONS
# =============================================================================

def build_url(endpoint, sample):
    """Build the DICOMWeb URL for retrieving a frame."""
    return f"{endpoint}/studies/{sample.study}/series/{sample.series}/instances/{sample.instance}/frames/{sample.frame}"

def run_curl_test(endpoint_name, endpoint_url, sample, header_name, header_value, use_auth=True):
    """
    Run a curl command and parse the response.
    Returns a dict with test results.
    """
    url = build_url(endpoint_url, sample)
    
    # Build curl command
    cmd = ["curl", "-s", "-w", "\n%{http_code}", "-X", "GET"]
    
    # Add authorization for Google Healthcare API endpoints
    if use_auth and "healthcare.googleapis.com" in endpoint_url:
        try:
            token = subprocess.run(
                ["gcloud", "auth", "print-access-token"],
                capture_output=True, text=True, check=True
            ).stdout.strip()
            cmd.extend(["-H", f"Authorization: Bearer {token}"])
        except subprocess.CalledProcessError:
            return {
                "endpoint": endpoint_name,
                "sample": sample.label,
                "accept_header": header_name,
                "status": "AUTH_ERROR",
                "http_code": None,
                "error_message": "Failed to get auth token",
                "success": False
            }
    
    # Add Accept header if provided
    if header_value:
        cmd.extend(["-H", f"Accept: {header_value}"])
    
    cmd.append(url)
    
    # Run the curl command
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)
        output = result.stdout
        
        # Parse HTTP status code (last line)
        lines = output.strip().split('\n')
        http_code = lines[-1] if lines else "000"
        body = '\n'.join(lines[:-1]) if len(lines) > 1 else ""
        
        # Determine success/failure and extract error message
        success = False
        error_message = None
        
        try:
            http_code_int = int(http_code)
        except ValueError:
            http_code_int = 0
        
        if http_code_int == 200:
            # Check if body contains error JSON
            if '"error"' in body:
                # Extract error message from JSON embedded in multipart response
                error_match = re.search(r'"message":\s*"([^"]+)"', body)
                if error_match:
                    error_message = error_match.group(1)
                    success = False
                else:
                    error_message = "Unknown error in response"
                    success = False
            else:
                success = True
        elif http_code_int >= 400:
            # Try to parse error from JSON response
            try:
                error_json = json.loads(body)
                error_message = error_json.get("error", {}).get("message", body[:200])
            except json.JSONDecodeError:
                error_message = body[:200] if body else f"HTTP {http_code}"
        else:
            error_message = f"Unexpected HTTP code: {http_code}"
        
        return {
            "endpoint": endpoint_name,
            "sample": sample.label,
            "accept_header": header_name,
            "status": "SUCCESS" if success else "FAILED",
            "http_code": http_code,
            "error_message": error_message,
            "success": success
        }
        
    except subprocess.TimeoutExpired:
        return {
            "endpoint": endpoint_name,
            "sample": sample.label,
            "accept_header": header_name,
            "status": "TIMEOUT",
            "http_code": None,
            "error_message": "Request timed out after 60s",
            "success": False
        }
    except Exception as e:
        return {
            "endpoint": endpoint_name,
            "sample": sample.label,
            "accept_header": header_name,
            "status": "ERROR",
            "http_code": None,
            "error_message": str(e),
            "success": False
        }

def run_test_matrix(endpoints, accept_headers, samples, verbose=True):
    """
    Run all combinations of endpoints, accept headers, and samples.
    Returns a list of test results.
    """
    results = []
    total_tests = len(endpoints) * len(accept_headers) * len(samples)
    current_test = 0
    
    print(f"Running test matrix: {len(endpoints)} endpoints x {len(accept_headers)} headers x {len(samples)} samples = {total_tests} tests\n")
    print("=" * 80)
    
    for endpoint_name, endpoint_url in endpoints.items():
        for header_name, header_value in accept_headers.items():
            for sample in samples:
                current_test += 1
                
                if verbose:
                    print(f"[{current_test}/{total_tests}] {endpoint_name} | {header_name} | {sample.label}...", end=" ")
                
                result = run_curl_test(endpoint_name, endpoint_url, sample, header_name, header_value)
                results.append(result)
                
                if verbose:
                    status_icon = "✓" if result["success"] else "✗"
                    print(f"{status_icon} {result['status']}")
    
    print("=" * 80)
    print(f"\nCompleted {total_tests} tests")
    
    return results

def generate_summary(results):
    """Generate a summary DataFrame and statistics from test results."""
    df = pd.DataFrame(results)
    
    # Create pivot table for quick overview
    print("\n" + "=" * 80)
    print("TEST RESULTS SUMMARY")
    print("=" * 80)
    
    # Overall statistics
    total = len(results)
    passed = sum(1 for r in results if r["success"])
    failed = total - passed
    
    print(f"\nOverall: {passed}/{total} passed ({100*passed/total:.1f}%)")
    print(f"         {failed}/{total} failed ({100*failed/total:.1f}%)")
    
    # Summary by endpoint
    print("\n--- By Endpoint ---")
    for endpoint in df["endpoint"].unique():
        endpoint_results = df[df["endpoint"] == endpoint]
        endpoint_passed = endpoint_results["success"].sum()
        endpoint_total = len(endpoint_results)
        print(f"  {endpoint}: {endpoint_passed}/{endpoint_total} passed")
    
    # Summary by sample
    print("\n--- By Sample ---")
    for sample in df["sample"].unique():
        sample_results = df[df["sample"] == sample]
        sample_passed = sample_results["success"].sum()
        sample_total = len(sample_results)
        print(f"  {sample}: {sample_passed}/{sample_total} passed")
    
    # Summary by accept header
    print("\n--- By Accept Header ---")
    for header in df["accept_header"].unique():
        header_results = df[df["accept_header"] == header]
        header_passed = header_results["success"].sum()
        header_total = len(header_results)
        print(f"  {header}: {header_passed}/{header_total} passed")
    
    # Show all failures with details
    failures = df[~df["success"]]
    if not failures.empty:
        print("\n--- Failed Tests ---")
        for _, row in failures.iterrows():
            print(f"  [{row['endpoint']}] [{row['accept_header']}] [{row['sample']}]")
            print(f"    HTTP: {row['http_code']} | Error: {row['error_message'][:80] if row['error_message'] else 'N/A'}...")
    
    # Create a cross-tabulation matrix
    print("\n--- Results Matrix (Sample x Endpoint) ---")
    pivot = df.pivot_table(
        index="sample",
        columns="endpoint",
        values="success",
        aggfunc=lambda x: f"{sum(x)}/{len(x)}"
    )
    print(pivot.to_string())
    
    return df

# =============================================================================
# RUN THE TEST MATRIX
# =============================================================================

print(f"Test Matrix Execution Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Testing {len(test_samples)} samples against {len(DICOMWEB_ENDPOINTS)} endpoints with {len(ACCEPT_HEADERS)} accept header configurations\n")

# Run all tests
test_results = run_test_matrix(DICOMWEB_ENDPOINTS, ACCEPT_HEADERS, test_samples, verbose=True)

# Generate and display summary
results_df = generate_summary(test_results)

print(f"\n\nTest Matrix Execution Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Test Matrix Execution Started: 2026-01-14 15:40:41
Testing 2 samples against 3 endpoints with 4 accept header configurations

Running test matrix: 3 endpoints x 4 headers x 2 samples = 24 tests

[1/24] IDC_PROXY | SLIM_FULL | failing slide... ✗ FAILED
[2/24] IDC_PROXY | SLIM_FULL | succeeding slide... ✗ ERROR
[3/24] IDC_PROXY | JPEG2000_ONLY | failing slide... ✗ FAILED
[4/24] IDC_PROXY | JPEG2000_ONLY | succeeding slide... ✗ ERROR
[5/24] IDC_PROXY | OCTET_STREAM | failing slide... ✗ ERROR
[6/24] IDC_PROXY | OCTET_STREAM | succeeding slide... ✗ ERROR
[7/24] IDC_PROXY | NO_HEADER | failing slide... ✗ FAILED
[8/24] IDC_PROXY | NO_HEADER | succeeding slide... ✗ ERROR
[9/24] GOOGLE_PUBLIC_v23 | SLIM_FULL | failing slide... ✗ FAILED
[10/24] GOOGLE_PUBLIC_v23 | SLIM_FULL | succeeding slide... ✗ ERROR
[11/24] GOOGLE_PUBLIC_v23 | JPEG2000_ONLY | failing slide... ✗ FAILED
[12/24] GOOGLE_PUBLIC_v23 | JPEG2000_ONLY | succeeding slide... ✗ ERROR
[13/24] GOOGLE_PUBLIC_v23 | OCTET_STREAM | failing sl

In [ ]:
%%bash
curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" \
${idc_internal_store}/studies/2.25.302737996345872783571112300080988167697/series/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0/instances/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0/frames/56


curl: (3) URL using bad/illegal format or missing URL


CalledProcessError: Command 'b'curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" \\\n"${idc_internal_store}/studies/2.25.302737996345872783571112300080988167697/series/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0/instances/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0/frames/56"\n'' returned non-zero exit status 3.

In [ ]:
!curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" \
https://healthcare.googleapis.com/v1/projects/canceridc-data/locations/us/datasets/idc/dicomStores/v23/dicomWeb/studies/2.25.302737996345872783571112300080988167697/series/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0/instances/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0/frames/56  \
-H 'Accept: multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.80, multipart/related; type="image/jls"; transfer-syntax=1.2.840.10008.1.2.4.81, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.90, multipart/related; type="image/jp2"; transfer-syntax=1.2.840.10008.1.2.4.91, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.92, multipart/related; type="image/jpx"; transfer-syntax=1.2.840.10008.1.2.4.93, multipart/related; type="application/octet-stream"; transfer-syntax=*, */*;q=0.1'


--39fd3842e38040f66733cd8999de0a015ad028f978c876fe0058e643111b
Content-Type: image/jls; transfer-syntax=1.2.840.10008.1.2.4.80

[{
  "error": {
    "code": 400,
    "message": "transcoding frame: generic::invalid_argument: Component 1 out of 3 components does not have the expected width. Component Width: 128, Expected Columns: 256",
    "status": "INVALID_ARGUMENT"
  }
}
]

In [ ]:
!
https://healthcare.googleapis.com/v1/projects/nci-idc-data/locations/us-central1/datasets/idc/dicomStores/idc-store-v23/dicomWeb

In [ ]:
!curl https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb/studies/2.25.302737996345872783571112300080988167697/series/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.2.0/instances/1.3.6.1.4.1.5962.99.1.1250863857.1162905243.1637633436401.29.0/frames/56
